# Bidirectional LSTM + CRF

If you are new to this post, then please refer to [Model Basics](https://github.com/akash1309/Named-Entity-Recognition/blob/master/Model_Basics.ipynb)

For this section, we will see a full, complicated example of a Bi-LSTM
Conditional Random Field for named-entity recognition. The LSTM tagger
above is typically sufficient for part-of-speech tagging, but a sequence
model like the CRF is really essential for strong performance on NER.
Familiarity with CRF's is assumed. Although this name sounds scary, all
the model is is a CRF but where an LSTM provides the features.  Recall that the CRF computes a conditional probability. Let
$y$ be a tag sequence and $x$ an input sequence of words.
Then we compute

\begin{align}P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})}\end{align}

Where the score is determined by defining some log potentials
$\log \psi_i(x,y)$ such that

\begin{align}\text{Score}(x,y) = \sum_i \log \psi_i(x,y)\end{align}

To make the partition function tractable, the potentials must look only
at local features.

In the Bi-LSTM CRF, we define two kinds of potentials: emission and
transition. The emission potential for the word at index $i$ comes
from the hidden state of the Bi-LSTM at timestep $i$. The
transition scores are stored in a $|T|x|T|$ matrix
$\textbf{P}$, where $T$ is the tag set. In my
implementation, $\textbf{P}_{j,k}$ is the score of transitioning
to tag $j$ from tag $k$. So:

\begin{align}\text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i)\end{align}

\begin{align}= \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}}\end{align}

where in this second expression, we think of the tags as being assigned
unique non-negative indices.


In [0]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [0]:
is_gpu = torch.cuda.is_available()
is_gpu

True

Helper functions to make the code more readable.

In [0]:
def argmax(vec):
  # return the argmax as a python int
  _, idx = torch.max(vec, 1)
  return idx.item()


def prepare_sequence(seq, to_ix):
  idxs = [to_ix[w] for w in seq]
  return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
  max_score = vec[0, argmax(vec)]
  max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
  return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

Create model

In [0]:
class BiLSTM_CRF(nn.Module):

  def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
    super(BiLSTM_CRF, self).__init__()
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.vocab_size = vocab_size
    self.tag_to_ix = tag_to_ix
    self.tagset_size = len(tag_to_ix)

    self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,num_layers=1, bidirectional=True)

    # Maps the output of the LSTM into tag space.
    self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Matrix of transition parameters.  Entry i,j is the score of
    # transitioning *to* i *from* j.
    self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

    # These two statements enforce the constraint that we never transfer
    # to the start tag and we never transfer from the stop tag
    self.transitions.data[tag_to_ix[START_TAG], :] = -10000
    self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

    self.hidden = self.init_hidden()

  def init_hidden(self):
    return (torch.randn(2, 1, self.hidden_dim // 2),torch.randn(2, 1, self.hidden_dim // 2))


  def _forward_alg(self, feats):
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, self.tagset_size), -10000.)
    # START_TAG has all of the score.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    # Wrap in a variable so that we will get automatic backprop
    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
      alphas_t = []  # The forward tensors at this timestep
      for next_tag in range(self.tagset_size):
        # broadcast the emission score: it is the same regardless of
        # the previous tag
        emit_score = feat[next_tag].view(
            1, -1).expand(1, self.tagset_size)
        # the ith entry of trans_score is the score of transitioning to
        # next_tag from i
        trans_score = self.transitions[next_tag].view(1, -1)
        # The ith entry of next_tag_var is the value for the
        # edge (i -> next_tag) before we do log-sum-exp
        next_tag_var = forward_var + trans_score + emit_score
        # The forward variable for this tag is log-sum-exp of all the
        # scores.
        alphas_t.append(log_sum_exp(next_tag_var).view(1))
      forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

  def _get_lstm_features(self, sentence):
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

  def _score_sentence(self, feats, tags):
    # Gives the score of a provided tag sequence
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
      score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

  def _viterbi_decode(self, feats):
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
      bptrs_t = []  # holds the backpointers for this step
      viterbivars_t = []  # holds the viterbi variables for this step

      for next_tag in range(self.tagset_size):
        # next_tag_var[i] holds the viterbi variable for tag i at the
        # previous step, plus the score of transitioning
        # from tag i to next_tag.
        # We don't include the emission scores here because the max
        # does not depend on them (we add them in below)
        next_tag_var = forward_var + self.transitions[next_tag]
        best_tag_id = argmax(next_tag_var)
        bptrs_t.append(best_tag_id)
        viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
      # Now add in the emission scores, and assign forward_var to the set
      # of viterbi variables we just computed
      forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
      backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
      best_tag_id = bptrs_t[best_tag_id]
      best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

  def neg_log_likelihood(self, sentence, tags):
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

  def forward(self, sentence):  # dont confuse this with _forward_alg above.
    # Get the emission scores from the BiLSTM
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

Run training

In [0]:
# For words
"""
word_filepath = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/words.txt'
word_to_ix = {}
with open(word_filepath,'r') as f:

  for i,word in enumerate(set(f.read().splitlines())):
    word_to_ix[word] = i   # Because first 2 indices are stored for padding and unknown character

#word_to_ix['<pad>'] = 0  # padding
#word_to_ix['UNK'] = 1    # unknown

ix_to_word = {index: word for word, index in word_to_ix.items()}
print(len(word_to_ix))
"""

"\nword_filepath = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/words.txt'\nword_to_ix = {}\nwith open(word_filepath,'r') as f:\n\n  for i,word in enumerate(set(f.read().splitlines())):\n    word_to_ix[word] = i   # Because first 2 indices are stored for padding and unknown character\n\n#word_to_ix['<pad>'] = 0  # padding\n#word_to_ix['UNK'] = 1    # unknown\n\nix_to_word = {index: word for word, index in word_to_ix.items()}\nprint(len(word_to_ix))\n"

In [0]:
# for tags.txt
"""
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tags_filepath = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/tags.txt'
tag_to_ix = {}

with open(tags_filepath,'r') as f:
  for i,word in enumerate(set(f.read().splitlines())):
    tag_to_ix[word] = i
tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)

ix_to_tag = {index: word for word, index in tag_to_ix.items()}
print(len(tag_to_ix))
"""

'\nSTART_TAG = "<START>"\nSTOP_TAG = "<STOP>"\ntags_filepath = \'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/tags.txt\'\ntag_to_ix = {}\n\nwith open(tags_filepath,\'r\') as f:\n  for i,word in enumerate(set(f.read().splitlines())):\n    tag_to_ix[word] = i\ntag_to_ix[START_TAG] = len(tag_to_ix)\ntag_to_ix[STOP_TAG] = len(tag_to_ix)\n\nix_to_tag = {index: word for word, index in tag_to_ix.items()}\nprint(len(tag_to_ix))\n'

In [0]:

EMBEDDING_DIM = 50
HIDDEN_DIM = 50

# Make up some training data
sent_file = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/sentences.txt'
label_file = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/labels.txt'
sentences = []
with open(sent_file,'r') as f:
  txt = f.read().splitlines()

for sent in txt:
  sentences.append(sent)

print(len(sentences))

labels = []
with open(label_file,'r') as f:
  txt = f.read().splitlines()

for lab in txt:
  labels.append(lab)

print(len(labels))
print(sentences[0])
print(labels[0])

data = [(sentences[i].split(), labels[i].split()) for i in range(0, len(sentences))]
# For now lets take only 1000 sentences

word_to_ix = {}
tag_to_ix = {}

for sentence, tags in data[:1010]:
  for word in sentence:
    if word not in word_to_ix:
      word_to_ix[word] = len(word_to_ix)

  for lab in tags:
    if lab not in tag_to_ix:
      tag_to_ix[lab] = len(tag_to_ix)
      
ix_to_word = {index: word for word, index in word_to_ix.items()}

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)

ix_to_tag = {index: word for word, index in tag_to_ix.items()}
print(len(word_to_ix),len(tag_to_ix))

training_data = data[:1000]
testing_data = data[1000:1010]
print(len(training_data))
#print(training_set[:2]) 
print(type(training_data))

47959
47959
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O
4675 19
1000
<class 'list'>


In [0]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model

BiLSTM_CRF(
  (word_embeds): Embedding(4675, 50)
  (lstm): LSTM(50, 25, bidirectional=True)
  (hidden2tag): Linear(in_features=50, out_features=19, bias=True)
)

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

In [0]:

# Check predictions before training
with torch.no_grad():
  precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
  precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
  print(model(precheck_sent))


(tensor(48.6656), [4, 13, 3, 11, 1, 14, 15, 1, 14, 15, 1, 14, 15, 1, 14, 4, 13, 3, 11, 3, 11, 10, 9, 8])


In [0]:
losses = []

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(30):  # again, normally you would NOT do 300 epochs, it is toy data
  tracker = 0
  for sentence, tags in training_data:
    tracker += 1
    # Step 1. Remember that Pytorch accumulates gradients.
    # We need to clear them out before each instance
    model.zero_grad()

    # Step 2. Get our inputs ready for the network, that is,
    # turn them into Tensors of word indices.
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
    
    # Step 3. Run our forward pass.
    loss = model.neg_log_likelihood(sentence_in, targets)
    losses.append(loss.item())
    if(tracker%500 == 0):
      print(f'Epochs : {epoch}  Loss Values : {loss.item()}')
    # Step 4. Compute the loss, gradients, and update the parameters by
    # calling optimizer.step()
    loss.backward()
    optimizer.step()



Epochs : 0  Loss Values : 3.600341796875
Epochs : 0  Loss Values : 1.7866630554199219
Epochs : 1  Loss Values : 2.4691009521484375
Epochs : 1  Loss Values : 3.413646697998047
Epochs : 2  Loss Values : 1.2697906494140625
Epochs : 2  Loss Values : 1.7527084350585938
Epochs : 3  Loss Values : 0.75506591796875
Epochs : 3  Loss Values : 0.8287887573242188
Epochs : 4  Loss Values : 0.5165252685546875
Epochs : 4  Loss Values : 0.480316162109375
Epochs : 5  Loss Values : 0.3390350341796875
Epochs : 5  Loss Values : 2.0801925659179688
Epochs : 6  Loss Values : 0.2626953125
Epochs : 6  Loss Values : 0.30590057373046875
Epochs : 7  Loss Values : 0.199493408203125
Epochs : 7  Loss Values : 0.24738311767578125
Epochs : 8  Loss Values : 0.20855712890625
Epochs : 8  Loss Values : 0.21897125244140625
Epochs : 9  Loss Values : 0.1334228515625
Epochs : 9  Loss Values : 0.07977294921875
Epochs : 10  Loss Values : 0.167388916015625
Epochs : 10  Loss Values : 0.15367889404296875
Epochs : 11  Loss Values : 

In [0]:
# Check predictions after training
with torch.no_grad():
  for sent, tag in testing_data:
    precheck_sent = prepare_sequence(sent, word_to_ix)
    a = model(precheck_sent)
    b = [ix_to_tag[t] for t in a[1]]
    print(f'Predicted Labels : {b}')
    print(f'Actual Labels :    {tag}')
    print('\n\n')  

Predicted Labels : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O']
Actual Labels :    ['O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['B-per', 'I-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O']
Actual Labels :    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']




Model Efficiency can be increased with hyperparameter tuning.
- No. of epochs 
- No. of hidden layers
- No. of hidden dimension
- Embedding size
- Learning Rate
- Other Features 

Now, we will be explaining different parts of the above code and logic.